In [ ]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.FileExtensions, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

In [ ]:
using System.IO;
using Microsoft.Extensions.Configuration;


var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile(Path.Combine(
        Environment.GetFolderPath(Environment.SpecialFolder.ApplicationData),
        "Microsoft",
        "UserSecrets",
        "2b6c897c-02c0-4c8e-91a8-cb61113a6acc", // comes from the project file QuokkaVec.Console.csproj
        "secrets.json")
    );

IConfigurationRoot configuration = builder.Build();
string awsAccessKeyId = configuration["AWSAccessKeyId"] ?? string.Empty;
string awsSecretAccessKey = configuration["AWSSecretAccessKey"] ?? string.Empty;

if (string.IsNullOrWhiteSpace(awsAccessKeyId) || string.IsNullOrWhiteSpace(awsSecretAccessKey))
{
	Console.WriteLine("AWSAccessKeyId or AWSSecretAccessKey is not set, use 'dotnet user-secrets init' and 'dotnet user-secrets set <key> <value>' to set the key, exiting...");
	return;
}

Console.WriteLine("API Keys set");

In [ ]:
bedrock = await import("https://esm.sh/@langchain/community/llms/bedrock");

// interesting why following is not needed
// npm install @aws-crypto/sha256-js @aws-sdk/credential-provider-node @smithy/protocol-http @smithy/signature-v4 @smithy/eventstream-codec @smithy/util-utf8 @aws-sdk/types

In [ ]:
#!set --name awsAccessKeyId --value @csharp:awsAccessKeyId
#!set --name awsSecretAccessKey --value @csharp:awsSecretAccessKey

In [ ]:
// If no credentials are provided, the default credentials from
// @aws-sdk/credential-provider-node will be used.
const model = new bedrock.Bedrock({
  model: "ai21.j2-grande-instruct", // You can also do e.g. "anthropic.claude-v2"
  region: "us-east-1",
  // endpointUrl: "custom.amazonaws.com",
  credentials: {
    accessKeyId: awsAccessKeyId,//process.env.BEDROCK_AWS_ACCESS_KEY_ID!,
    secretAccessKey: awsSecretAccessKey, //process.env.BEDROCK_AWS_SECRET_ACCESS_KEY!,
  },
  // modelKwargs: {},
});

const res = await model.invoke("Tell me a joke");
console.log(res);

Note on JS docs: https://js.langchain.com/v0.1/docs/integrations/text_embedding/bedrock/


In [ ]:
bedrockEmbeddings = await import("https://esm.sh/@langchain/community/embeddings/bedrock");
bedrockClient = await import("https://esm.sh/@aws-sdk/client-bedrock-runtime");

console.log('Creating embeddings model');
const embeddings = new bedrockEmbeddings.BedrockEmbeddings({
  client: new bedrockClient.BedrockRuntimeClient({
    region: "us-east-1",
    credentials: {
      accessKeyId: awsAccessKeyId,
      secretAccessKey: awsSecretAccessKey,
    }
  }),
  // region: "us-east-1",
  // credentials: {
  //   accessKeyId: awsAccessKeyId,
  //   secretAccessKey: awsSecretAccessKey,
  // },
  model: 'amazon.titan-embed-text-v2:0',//"amazon.titan-embed-text-v1",
});
console.log('Embeddings model created');

console.log('Embedding query');
const res = await embeddings.embedQuery(
  "What would be a good company name a company that makes colorful socks?"
);
console.log('Embedding query done');
console.log({ res });



# Store embedding result (res) into local qdrant container

In [ ]:
qdrant = await import("https://esm.sh/@langchain/qdrant");

In [ ]:
bedrockEmbeddings = await import("https://esm.sh/@langchain/community/embeddings/bedrock");
bedrockClient = await import("https://esm.sh/@aws-sdk/client-bedrock-runtime");
qdrant = await import("https://esm.sh/@langchain/qdrant");

bedrockEmbeddings = new bedrockEmbeddings.BedrockEmbeddings({
  client: new bedrockClient.BedrockRuntimeClient({
    region: "us-east-1",
    credentials: {
      accessKeyId: awsAccessKeyId,
      secretAccessKey: awsSecretAccessKey,
    }
  }),
  model: 'amazon.titan-embed-text-v2:0',//"amazon.titan-embed-text-v1",
});

vectorStore = await qdrant.QdrantVectorStore.fromTexts(
  [
    'My code is Quokka',
    'Quokka is a great',
    'I use QDrant and Bedrock',
  ],
  [{ id: 1 }, { id: 2 }, { id: 3 }],
  bedrockEmbeddings,
  {
    url: "http://localhost:6333",
    collectionName: "test-langchain-js",
  }
);

const response = await vectorStore.similaritySearch("What tools I use?", 2);
console.log(response);

return response;

# Retriever
https://js.langchain.com/v0.1/docs/integrations/retrievers/vectorstore/
https://js.langchain.com/v0.1/docs/integrations/retrievers/bedrock-knowledge-bases/

In [ ]:
retriever = vectorStore.asRetriever();
const response = retriever.invoke("What tools I use?")
console.log(response);

return response;

TODO: test
https://js.langchain.com/v0.1/docs/integrations/vectorstores/memory/
https://js.langchain.com/v0.1/docs/integrations/retrievers/bedrock-knowledge-bases/
https://js.langchain.com/v0.1/docs/integrations/tools/lambda_agent/